In [ ]:
# 해당 파일은 협업 피터링으로 임베딩을 학습 후 컨텐츠 기반 필터리으로 유사도 기반 추천을 수행하는 추천 엔진

In [1]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [2]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
# 0 부터 시작하기 위해(데이터는 1 부터 시작하기 때문)
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [4]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1,n_movies+1)
shape

(943, 1682)

In [7]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

In [8]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [21]:
adj_matrix.shape

(943, 1682)

In [17]:
# U: 사용자, S: 특이 값 ,V: 항목
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [18]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [23]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

In [26]:
U

array([[ 0.06580431,  0.00597507],
       [ 0.01402104, -0.04662601],
       [ 0.00565798, -0.02561846],
       ...,
       [ 0.00744452, -0.02502129],
       [ 0.02403119,  0.0080961 ],
       [ 0.04224209, -0.01092715]])